# ตัวอย่าง model

In [ ]:
import os
from django.db import models
from django.contrib.auth.models import User
from django.utils import timezone

# โมเดลเสริม (1) สำหรับ OneToOneField
class UserProfile(models.Model):
    user = models.OneToOneField(User, on_delete=models.CASCADE, related_name="profile")
    bio = models.TextField(blank=True)
    profile_pic = models.ImageField(upload_to='profile_pics/', blank=True, null=True)

    def __str__(self):
        return self.user.username

# โมเดลเสริม (2) สำหรับ ManyToManyField
class Tag(models.Model):
    name = models.CharField(max_length=50, unique=True)

    def __str__(self):
        return self.name

# โมเดลหลัก "Kitchen Sink" ของเรา
class ComprehensiveRecord(models.Model):
    # --- Text Fields ---
    title = models.CharField(max_length=200, help_text="หัวข้อหลัก")
    description = models.TextField(blank=True, help_text="รายละเอียดแบบยาว")

    # --- Number Fields ---
    item_count = models.IntegerField(default=0, help_text="จำนวนเต็ม")
    temperature = models.FloatField(null=True, blank=True, help_text="เลขทศนิยม (ไม่แม่นยำสูง)")
    price = models.DecimalField(max_digits=10, decimal_places=2, help_text="เลขทศนิยม (แม่นยำสูง)")

    # --- Boolean Field ---
    is_published = models.BooleanField(default=False, help_text="สถานะเผยแพร่ (True/False)")

    # --- Date & Time Fields ---
    publish_date = models.DateField(null=True, blank=True, help_text="วันที่เผยแพร่ (เฉพาะวัน)")
    publish_time = models.TimeField(null=True, blank=True, help_text="เวลาที่เผยแพร่ (เฉพาะเวลา)")
    last_modified = models.DateTimeField(auto_now=True, help_text="แก้ไขล่าสุด (อัตโนมัติ)")
    created_at = models.DateTimeField(auto_now_add=True, help_text="สร้างเมื่อ (อัตโนมัติครั้งเดียว)")

    # --- Specialized Text Fields ---
    contact_email = models.EmailField(blank=True)
    website = models.URLField(blank=True)
    
    # --- File Fields (ต้องตั้งค่า MEDIA_ROOT) ---
    attachment = models.FileField(upload_to='attachments/', blank=True, null=True)

    # --- Relationship Fields ---
    # OneToOne ถูกลิงก์มาจาก UserProfile
    owner = models.ForeignKey(UserProfile, on_delete=models.CASCADE, related_name="records")
    tags = models.ManyToManyField(Tag, blank=True, related_name="records")

    def __str__(self):
        return self.title

# crud

## setup (คือไรไม่รู้)

In [ ]:
# --- SETUP ---
from yourapp.models import UserProfile, Tag, ComprehensiveRecord
from django.contrib.auth.models import User
from datetime import date, time
from decimal import Decimal
from django.utils import timezone

# 1. สร้าง User และ UserProfile (สำหรับ ForeignKey & OneToOneField)
# (สมมติว่ามี user ที่ username='testuser' อยู่แล้ว ถ้าไม่มีให้สร้างก่อน)
test_user, created = User.objects.get_or_create(username='testuser')
profile, created = UserProfile.objects.get_or_create(user=test_user, defaults={'bio': 'This is a test profile.'})

# 2. สร้าง Tags (สำหรับ ManyToManyField)
# วิธียัดข้อมูลสำหรับ many to many
tag1, created = Tag.objects.get_or_create(name='Python')
tag2, created = Tag.objects.get_or_create(name='Django')
tag3, created = Tag.objects.get_or_create(name='Tutorial')

print("Setup Complete!")

## create (c)

In [ ]:
#### **C - CREATE (การสร้าง)**

# --- CREATE ---

# สร้างอ็อบเจกต์ด้วย .create()
# **สังเกตชนิดข้อมูลที่ส่งเข้าไปในแต่ละ Field**
rec1 = ComprehensiveRecord.objects.create(
    # Text
    title="My First Comprehensive Record",
    description="This record includes various field types.",
    
    # Numbers
    item_count=10,
    temperature=36.6,
    price=Decimal("199.99"),  # **สำคัญมาก: DecimalField ต้องใช้ Decimal หรือ string**

    # Boolean
    is_published=True,

    # Date & Time (แบบกำหนดเอง)
    publish_date=date(2024, 1, 15),
    publish_time=time(10, 30, 0),

    # Specialized Text
    contact_email="test@example.com",
    website="https://djangoproject.com",

    # File (ใน shell เราจะใส่แค่ path ที่สมมติขึ้น)
    attachment="path/to/my/file.pdf",

    # Relationship (ForeignKey)
    # **สำคัญ: ต้องส่ง "อ็อบเจกต์" เข้าไป ไม่ใช่ ID**
    owner=profile 
)

# **สำคัญ: ManyToManyField ไม่สามารถเพิ่มตอน .create() ได้ ต้องทำหลังสร้างเสร็จ**
rec1.tags.add(tag1, tag2)
# .tags คือ ตัวแปรที่อยู่ใน class นะ ถ้าเป็นชื่ออื่นก็เปลี่ยนตาม

print(f"Created Record: '{rec1.title}' with ID: {rec1.id}")
print(f"Tags added: {[tag.name for tag in rec1.tags.all()]}")

## read (r)

In [ ]:
# --- READ ---

# 1. Get: อ่านแบบเจาะจง (ถ้าไม่เจอจะ error)
record = ComprehensiveRecord.objects.get(id=1)
print(f"GET >> Title: {record.title}, Price: {record.price}")

# 2. All: อ่านทั้งหมด
all_records = ComprehensiveRecord.objects.all()
print(f"ALL >> Found {all_records.count()} records.")

# 3. Filter: กรองด้วยเงื่อนไขต่างๆ
#   - ค้นหาจาก BooleanField
published_records = ComprehensiveRecord.objects.filter(is_published=True)

#   - ค้นหาจาก NumberField (ใช้ __gt, __lt, __gte, __lte)
expensive_records = ComprehensiveRecord.objects.filter(price__gte=Decimal("100.00"))

#   - ค้นหาจาก DateField (ใช้ __year, __month, __day)
records_from_2024 = ComprehensiveRecord.objects.filter(publish_date__year=2024)

#   - ค้นหาจาก CharField (ใช้ __icontains, __startswith)
records_with_first = ComprehensiveRecord.objects.filter(title__icontains="first")

#   - ค้นหาข้ามความสัมพันธ์ (ใช้ __)
#     หา Record ทั้งหมดที่เจ้าของ (owner) คือ 'testuser'
records_by_user = ComprehensiveRecord.objects.filter(owner__user__username='testuser')

#     หา Record ทั้งหมดที่มี Tag ชื่อ 'Python'
python_records = ComprehensiveRecord.objects.filter(tags__name='Python')

print(f"FILTER >> Found {python_records.count()} records with 'Python' tag.")

## update (u)

In [ ]:
# --- UPDATE ---

# 1. อ่านอ็อบเจกต์ที่ต้องการแก้ไขออกมาก่อน
rec_to_update = ComprehensiveRecord.objects.get(id=1)
print(f"BEFORE UPDATE >> Title: '{rec_to_update.title}', Stock: {rec_to_update.item_count}")

# 2. แก้ไขค่าในแต่ละ Field
rec_to_update.title = "My Updated Record Title" # CharField
rec_to_update.is_published = False             # BooleanField
rec_to_update.item_count += 5                  # IntegerField
rec_to_update.publish_date = None              # DateField (ทำให้เป็นค่าว่าง)

# 3. สั่งบันทึกการเปลี่ยนแปลง
rec_to_update.save()

print(f"AFTER UPDATE >> Title: '{rec_to_update.title}', Stock: {rec_to_update.item_count}")

# --- การแก้ไข ManyToManyField ---
#   - การลบ Tag
rec_to_update.tags.remove(tag1)
print(f"TAGS AFTER REMOVE >> {[tag.name for tag in rec_to_update.tags.all()]}")

#   - การเพิ่ม Tag ใหม่
rec_to_update.tags.add(tag3)
print(f"TAGS AFTER ADD >> {[tag.name for tag in rec_to_update.tags.all()]}")

#   - การเคลียร์ทั้งหมดแล้วตั้งค่าใหม่ (set)
rec_to_update.tags.set([tag1, tag3])
print(f"TAGS AFTER SET >> {[tag.name for tag in rec_to_update.tags.all()]}")

## delete (d)

In [ ]:
# --- DELETE ---

# 1. อ่านอ็อบเจกต์ที่ต้องการลบ
rec_to_delete = ComprehensiveRecord.objects.get(id=1)
print(f"Preparing to delete record with ID: {rec_to_delete.id}")

# 2. สั่งลบ
result = rec_to_delete.delete()
print(f"Deletion result: {result}")

# 3. ตรวจสอบว่ามันหายไปจริง
try:
    ComprehensiveRecord.objects.get(id=1)
except ComprehensiveRecord.DoesNotExist:
    print("Record with ID 1 no longer exists. Deletion successful.")

---

# models.py ตัวอย่าง ใช้กับ annotate() + aggregate()

In [ ]:
# models.py
from django.db import models

class Publisher(models.Model):
    name = models.CharField(max_length=200)

class Author(models.Model):
    name = models.CharField(max_length=100)

class Book(models.Model):
    title = models.CharField(max_length=200)
    publication_date = models.DateField()
    price = models.DecimalField(max_digits=8, decimal_places=2)
    rating = models.FloatField(null=True, blank=True)
    author = models.ForeignKey(Author, on_delete=models.CASCADE, related_name='books')
    publisher = models.ForeignKey(Publisher, on_delete=models.CASCADE, related_name='books')

In [ ]:
from django.db.models import F, Sum, Avg, Count, Max, Min, Case, When, Value, Q
from django.db.models import F, DecimalField
from django.db.models.functions import ExtractYear, Concat


# annotate()
พวก sum ก็คือ ทำแค่ตัวที่ row ซ้ำกัน ไม่เหมือน aggregate ที่ sum ทั้งหมดในตาราง  
ข้างล่างนี้คือใช้ format แล้วแก้เป็น sum count min max อะไรก็ได้ตามใจ

## count

In [ ]:
# views.py
authors = Author.objects.annotate(
    num_books=Count('books') # 'books' คือ related_name จาก Book model
)

for author in authors:
    print(f"นักเขียน: {author.name}, จำนวนหนังสือ: {author.num_books}")
# ผลลัพธ์:
# นักเขียน: J.K. Rowling, จำนวนหนังสือ: 7
# นักเขียน: George R.R. Martin, จำนวนหนังสือ: 5

## max

In [ ]:
publishers = Publisher.objects.annotate(
    max_book_price=Max('books__price') # ใช้ '__' เพื่อข้าม relationship
)

for publisher in publishers:
    print(f"สำนักพิมพ์: {publisher.name}, ราคาสูงสุด: {publisher.max_book_price}")

## avg

In [ ]:
authors = Author.objects.annotate(
    average_rating=Avg('books__rating')
)

for author in authors:
    print(f"นักเขียน: {author.name}, เรตติ้งเฉลี่ย: {author.average_rating:.2f}")

## F คูณอะไรสักอย่าง

In [ ]:
from django.db.models import F, DecimalField

books = Book.objects.annotate(
    discounted_price=F('price') * 0.9
)

for book in books:
    print(f"หนังสือ: {book.title}, ราคาปกติ: {book.price}, ราคาหลังหักส่วนลด: {book.discounted_price}")

## F +- ต่าง ๆ

In [ ]:
books = Book.objects.annotate(
    profit_margin=F('price') - (F('price') * 0.3)
)

for book in books:
    print(f"หนังสือ: {book.title}, ราคา: {book.price}, กำไร: {book.profit_margin}")

## case (จารย์ไม่ได้สอน)

In [ ]:
from django.db.models import Case, When, Value

books = Book.objects.annotate(
    price_category=Case(
        When(price__lte=300, then=Value('ราคาถูก')),
        When(price__gt=700, then=Value('ราคาแพง')),
        default=Value('ราคากลาง'),
        # output_field=CharField() # สามารถระบุประเภทของฟิลด์ได้
    )
)

for book in books:
    print(f"หนังสือ: {book.title}, ประเภทราคา: {book.price_category}")

## Q (filter ไรงี้)

In [ ]:
from django.db.models import Q

authors = Author.objects.annotate(
    num_highly_rated_books=Count('books', filter=Q(books__rating__gte=4.5))
)

for author in authors:
    print(f"นักเขียน: {author.name}, จำนวนหนังสือเรตติ้งดี: {author.num_highly_rated_books}")

## extract year

In [ ]:
from django.db.models.functions import ExtractYear

books = Book.objects.annotate(
    publication_year=ExtractYear('publication_date')
)

for book in books:
    print(f"หนังสือ: {book.title}, ปีที่ตีพิมพ์: {book.publication_year}")

## concat

In [ ]:
from django.db.models.functions import Concat
from django.db.models import Value

books = Book.objects.annotate(
    full_description=Concat(
        'title', Value(' (เขียนโดย '), 'author__name', Value(')'),
        # output_field=CharField()
    )
)

for book in books:
    print(book.full_description)
# ผลลัพธ์:
# Harry Potter and the Sorcerer's Stone (เขียนโดย J.K. Rowling)

## annotate().filter().order_by()

In [ ]:
prolific_authors = Author.objects.annotate(
    num_books=Count('books')
).filter(
    num_books__gt=5  # กรองด้วยฟิลด์ 'num_books' ที่เราเพิ่งสร้าง
).order_by(
    '-num_books'     # เรียงจากมากไปน้อย
)

for author in prolific_authors:
    print(f"นักเขียน: {author.name}, จำนวนหนังสือ: {author.num_books}")

## annotate() ซ้อน ๆ กัน .annotate().annotate()

In [ ]:
authors = Author.objects.annotate(
    num_books=Count('books')
).annotate(
    author_status=Case(
        When(num_books__gt=10, then=Value('นักเขียนมืออาชีพ')),
        When(num_books__gt=5, then=Value('นักเขียนผลงานเยอะ')),
        default=Value('นักเขียนทั่วไป')
    )
)

for author in authors:
    print(f"นักเขียน: {author.name}, สถานะ: {author.author_status}")

---
# aggregate()

In [ ]:
from django.db.models import Count, Avg, Max, Min, Sum

# คำนวณค่าสรุปจาก Model Book ทั้งหมด
book_summary = Book.objects.aggregate(
    total_books=Count('id'),          # นับจำนวนหนังสือทั้งหมด
    average_price=Avg('price'),       # หาราคาเฉลี่ย
    cheapest_book=Min('price'),       # หาราคาที่ถูกที่สุด
    most_expensive_book=Max('price'), # หาราคาที่แพงที่สุด
    total_value=Sum('price')          # หาราคาทั้งหมดรวมกัน
)

# ผลลัพธ์ที่ได้คือ Dictionary
print(book_summary)
# ผลลัพธ์ที่อาจจะได้:
# {
#   'total_books': 58,
#   'average_price': Decimal('499.50'),
#   'cheapest_book': Decimal('120.00'),
#   'most_expensive_book': Decimal('1500.00'),
#   'total_value': Decimal('28971.00')
# }

print(f"มีหนังสือในระบบทั้งหมด: {book_summary['total_books']} เล่ม")
print(f"ราคาเฉลี่ยอยู่ที่: {book_summary['average_price']:.2f} บาท")

In [ ]:
# นับ ID ของ author ที่ไม่ซ้ำกันในตาราง Book
distinct_authors_count = Book.objects.aggregate(
    num_authors=Count('author', distinct=True)
)

print(distinct_authors_count)
# ผลลัพธ์ที่อาจจะได้: {'num_authors': 15}
print(f"มีนักเขียนที่มีผลงานตีพิมพ์ทั้งหมด (ไม่ซ้ำ): {distinct_authors_count['num_authors']} คน")

In [ ]:
from django.db.models import Q

highly_rated_books_value = Book.objects.aggregate(
    total_value=Sum('price', filter=Q(rating__gte=4.5))
)

print(highly_rated_books_value)
# ผลลัพธ์ที่อาจจะได้: {'total_value': Decimal('12500.75')}
print(f"มูลค่ารวมของหนังสือเรตติ้งดี: {highly_rated_books_value['total_value']}")

In [ ]:
modern_books_count = Book.objects.aggregate(
    count=Count('id', filter=Q(publication_date__year__gt=2010))
)

print(modern_books_count)
# ผลลัพธ์ที่อาจจะได้: {'count': 35}
print(f"จำนวนหนังสือที่ตีพิมพ์หลังปี 2010: {modern_books_count['count']}")

In [ ]:
from django.db.models import F

total_profit = Book.objects.aggregate(
    total_profit=Sum(F('price') * 0.7)
)

print(total_profit)
# ผลลัพธ์ที่อาจจะได้: {'total_profit': 20279.7}
print(f"กำไรรวมโดยประมาณ: {total_profit['total_profit']:.2f} บาท")

## การใช้ values().annotate() (วิธีที่แนะนำ)

In [ ]:
# คำนวณจำนวนหนังสือของแต่ละสำนักพิมพ์
publisher_counts = Publisher.objects.annotate(num_books=Count('books')).values('name', 'num_books')
# ผลลัพธ์จะเป็น QuerySet ของ dictionaries:
# <QuerySet [{'name': 'Penguin', 'num_books': 10}, {'name': 'Bloomsbury', 'num_books': 7}, ...]>

# การใช้ values().aggregate() (ไม่ค่อยใช้ และอาจทำให้สับสน)
data = Book.objects.values('publisher__name').aggregate(total_books=Count('id'))
# ผลลัพธ์ที่ได้อาจจะเป็น: {'total_books': 58} ซึ่งคือการนับรวมทั้งหมดอยู่ดี ไม่ได้แยกกลุ่ม
# พฤติกรรมอาจไม่เป็นไปตามที่คาดหวังเสมอไปในกรณีนี้